<a href="https://colab.research.google.com/github/nicholasfurl/Great-Courses/blob/main/Lesson15_vgg16CNN_bugs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##########
#-----vgg16 DCNN bug versus animal classification
#This notebook trains an image classification model and makes gradual improvements, 
#debugging the model, to improve performance. GPUs are encouraged. 
#In colab, one can add a GPU by clicking the Runtime menu and selecting Change runtime type. 
#Selecting GPU as the hardware accelerator will allow for the usage of a GPU.
#########

In [1]:
#Download tinyImageNet
#200 categories, 500 images per category, images 64*64*3

#unzipping creates a tiny-imagenet-200 folder in active directory with test, train and val subfolders and two *.txt files
#Train subfolders have lots of further subfolders, eacj labveled n10*. Within each is some text files and an images folder
#containing n*.JPEG images
#Test subfolder has subfolder images with test_*JPEG files
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

--2023-01-03 16:30:48--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  20.1MB/s    in 15s     

2023-01-03 16:31:03 (16.0 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
!pip install keras=='2.8.0'
#from PIL import Image    #I knpow from previous lessons that this doesn't work
from keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#specify folder names  (for training set?)
cats_0 = ['n01443537','n01629819','n01641577','n01644900','n01698640','n01742172',
          'n01855672','n01882714','n02002724','n02056570','n02058221','n02074367',
          'n02085620','n02094433','n02099601','n02099712','n02106662','n02113799',
          'n02123045','n02123394','n02124075','n02125311','n02129165','n02132136',
          'n02364673','n02395406','n02403003','n02410509','n02415577','n02423022',
          'n02437312','n02480495','n02481823','n02486410','n02504458','n02509815']
cats_1 = ['n01770393','n01774384','n01774750','n01784675','n02165456','n02190166',
          'n02206856','n02226429','n02231487','n02233338','n02236044','n02268443',
          'n02279972','n02281406']

In [ ]:
#function def to read files and return training and test sets

#the cats argument will be one of the 
def read_cats(cats, lab, train_size, test_size):

  #initialise stuff for loop
  vecs = []
  labs = []

  num_images_per_folder = 500 #a bit more explanatory and robust than it was

  #loop through each folder name (a subcategory of animals or bugs) and then through the images in that folder
  for c in cats:  #folder names from argument
    for i in range(num_images_per_folder): #num images per folder`

    #load the image and convert it to an array
      img = image.load_img("tiny-imagenet-200/train/"+c+"/images/"+c+"_"+str(i)+".JPEG")
      img_arr = image.img_to_array(img)

      
      img_arr = img_arr.flatten()
      img_arr = img_arr.reshape(64,64,3)
      vecs += [img_arr]
      labs += [lab]
  vecs = np.asarray(vecs)
  return(train_test_split(vecs,labs, train_size=train_size,test_size=test_size))